In [22]:
from google.cloud import bigquery
import configparser
import os
import pandas as pd

config = configparser.ConfigParser()
config.read('config.ini')
latency_table_id = config['TABLES']['latency_table_id']
agent_table_id = config['TABLES']['agent_table_id']
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config['SECRETS']['service_key_path']

bqclient = bigquery.Client()

# Download query results.
query_string = """
SELECT * FROM `dave-selfstudy01.wtrace.tx_result`
WHERE
dest_url = "https://coscloud.99.com/43f2bdd810c750ec7b7fedee2d1219fa/dcloud_default_component.zip"
AND datetime > '2022-01-28T10:40:07.293206';
"""

dataframe = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(
        # Optionally, explicitly request to use the BigQuery Storage API. As of
        # google-cloud-bigquery version 1.26.0 and above, the BigQuery Storage
        # API is used by default.
        create_bqstorage_client=True,
    )
)
df = dataframe
df.head()

,run_id,agent_ip,agent_asn,isp,metro,remote_ip,remote_ip_info,dns_latency_us,ping_min_latency_ms,ping_avg_latency_ms,...,tcp_latency_us,http_latency_us,response_code,response_length,country_code,datetime,gcp_pop,age,loop,dest_url
0,20220128185139,103.11.1.40,136525,wancompk,khi2,34.107.138.156,cloud:15169:global:CLOUD-PREMIUM-V4,325675,19.394,19.455,...,19683,1323287,200,186447021,PK,2022-01-28 10:51:42.016309,None,804,1,https://coscloud.99.com/43f2bdd810c750ec7b7fed...
1,20220128185139,111.119.185.132,59257,zong,lhe4,34.107.138.156,cloud:15169:global:CLOUD-PREMIUM-V4,332442,33.779,33.958,...,34767,1504702,200,186447021,PK,2022-01-28 10:51:42.016309,None,804,1,https://coscloud.99.com/43f2bdd810c750ec7b7fed...
2,20220128185139,103.217.177.5,135407,tespk,khi2,34.107.138.156,cloud:15169:global:CLOUD-PREMIUM-V4,618982,31.304,31.493,...,32657,1876404,200,186447021,PK,2022-01-28 10:51:42.016309,None,804,1,https://coscloud.99.com/43f2bdd810c750ec7b7fed...
3,20220128185139,103.29.163.196,58893,gemnetpk,hdd1,34.107.138.156,cloud:15169:global:CLOUD-PREMIUM-V4,632309,15.340,15.560,...,16267,1591952,200,186447021,PK,2022-01-28 10:51:42.016309,None,804,1,https://coscloud.99.com/43f2bdd810c750ec7b7fed...
4,20220128185139,202.163.115.135,9541,cyber,khi4,34.107.138.156,cloud:15169:global:CLOUD-PREMIUM-V4,675990,12.306,12.566,...,13474,1462332,200,186447021,PK,2022-01-28 10:51:42.016309,None,804,1,https://coscloud.99.com/43f2bdd810c750ec7b7fed...


In [23]:
(df.response_length/df.http_latency_us)[df.response_code == 200].describe()

count    208.000000
mean     102.022022
std       35.662780
min        6.297673
25%       88.162551
50%      117.193852
75%      125.098707
max      142.614138
dtype: float64

In [24]:
len(df.agent_ip.unique())

71

In [25]:
len(df.isp.unique())

28